In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import PIL
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import Sample_AlexNet
from SampleNN_Print import *
from tqdm import tqdm_notebook
%matplotlib inline
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
import torchvision

In [2]:
import torchvision.transforms as transforms
traindir = '/home/yanzy/data/train'
valdir = '/home/yanzy/data/val'
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

val_dataset =  torchvision.datasets.ImageFolder(
    valdir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
]))
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1,
                                         shuffle=False, num_workers = 32, pin_memory=True)

In [4]:
def val(Sample = 1, N = 16, m = 8):
    TMP = protectStateDict(sample_net)
    sampleStateDict(sample_net, N, m)
    sample_net.eval()
    correct = 0
    total = 0
    count = 0
    with torch.no_grad():
        for data in tqdm_notebook(val_loader, desc = "m = %d"%(m), leave = False):
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                images = mSample(N=N,m=m)(images)
                outputs = sample_net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

    print('When m = %d, Accuracy of the network on the %d test images: %.3f %%' % (m, total,
        100.0 * correct / total))
    
    sample_net.load_state_dict(TMP)
    del TMP
    return 100.0 * correct / total

In [5]:
def val_float(Sample = 1, N = 16, m = 8):
    float_net.eval()
    correct = 0
    total = 0
    count = 0
    with torch.no_grad():
        for data in tqdm_notebook(val_loader, desc = "m = %d"%(m), leave = False):
            
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = float_net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)


                correct += (predicted == labels).sum().item()
                #print correct, total

    print('When m = %d, Accuracy of the network on the %d test images: %.3f %%' % (m, total,
        100.0 * correct / total))
    
    return 100.0 * correct / total

## With Clamp version

In [3]:
float_net = torchvision.models.alexnet(pretrained=True).to(device)
gt  = float_net.state_dict()
b = gt.keys()
N = 16
m = 8

In [4]:
sample_net = Sample_AlexNet.alexnet().to(device)
lol = sample_net.state_dict()
a = lol.keys()
for i in range(len(a)):
    lol[a[i]] = gt[b[i]]
    print (lol[a[i]].shape)
sample_net.load_state_dict(lol)
sampleStateDict(sample_net, N, m)
sample_net.eval()
print (len(a))

torch.Size([64, 3, 11, 11])
torch.Size([64])
torch.Size([192, 64, 5, 5])
torch.Size([192])
torch.Size([384, 192, 3, 3])
torch.Size([384])
torch.Size([256, 384, 3, 3])
torch.Size([256])
torch.Size([256, 256, 3, 3])
torch.Size([256])
torch.Size([4096, 9216])
torch.Size([4096])
torch.Size([4096, 4096])
torch.Size([4096])
torch.Size([1000, 4096])
torch.Size([1000])
16


In [7]:
val()

When m = 8, Accuracy of the network on the 50000 test images: 44.354 %


44.354

In [7]:
import time
happysfsd = 0
with torch.no_grad():
    for data in tqdm_notebook(val_loader, desc = "m = %d"%(m), leave = False):
            images, labels = data
            if labels == 23:
                happysfsd = happysfsd + 1
            if happysfsd == 13:
                break

In [8]:
print torch.max(sample_net(mSample(N=16,m=8)(images).to(device)).data, 1)
print labels

(tensor([14.3750], device='cuda:0'), tensor([138], device='cuda:0'))
tensor([23])


In [76]:
M = (images.to(torch.float32)/pow(2,-8)).round().clamp(-pow(2,16),pow(2,16)-1).to(torch.int32)


In [8]:
a = torch.load('./Weights/features.0.weight.pt')
print(a.size())

torch.Size([64, 3, 11, 11])
